In [15]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from sklearn import svm
import os,shutil

In [35]:
import pytesseract as tess
from tqdm import tqdm

In [2]:
'''
显示图片
输入：图片矩阵，画布大小：元组（宽，高）
'''
def showImg(img,figsize=(4,4),cmap = "gray"):
    plt.figure(figsize=figsize)
    plt.axis('off')
    plt.imshow(img,cmap = cmap)
    plt.show()

In [3]:
'''
读取文件夹下的所有文件及图片，以灰度图的形式读取并resize到指定大小
输入：input_dir：读取的文件夹，可使用模糊匹配,Rmap表示读取的形式，若为0则以灰度图形式读取
输出：图像列表，图片的路径列表
'''
def readImgAPath(input_dir,imgsize=32,Rmap=0):
    glob_dir = input_dir + '*.png'
    if Rmap==0:
        images = [cv.resize(cv.imread(file,0),(imgsize,imgsize)) for file in glob.glob(glob_dir)] #通过通配符读取图像文件并且进行resize
    else:
        images = [cv.resize(cv.imread(file),(imgsize,imgsize)) for file in glob.glob(glob_dir)] #通过通配符读取图像文件并且进行resize
    paths = [file for file in glob.glob(glob_dir)] #读取对应的路径
    # images = np.array(images)/255 #将一张图flatten之后归一化
    return images,paths

In [4]:
'''
读取文件夹下的所有文件及图片，以灰度图的形式读取并resize到指定大小并且进行norm
输入：input_dir：读取的文件夹，可使用模糊匹配
输出：图像列表，图片的路径列表
'''
def readImgAPathWithNorm(input_dir,imgsize=32,Rmap=0):
    glob_dir = input_dir + '*.png'
    if Rmap==0:
        images = [cv.resize(cv.imread(file,0),(imgsize,imgsize)) for file in glob.glob(glob_dir)] #通过通配符读取图像文件并且进行resize
    else:
        images = [cv.resize(cv.imread(file),(imgsize,imgsize)) for file in glob.glob(glob_dir)] #通过通配符读取图像文件并且进行resize
    paths = [file for file in glob.glob(glob_dir)] #读取对应的路径
    images = np.array(images)/255 #将一张图flatten之后归一化
    return images,paths

In [5]:
# #读取指定文件夹下的所有图片并且resize为224*224 在将图像数据归一化并且通过reshape扁平化
# input_dir = "./result/mask_test/?/"
# glob_dir = input_dir + '*.png'
# images = [cv.resize(cv.imread(file,0),(32,32)) for file in glob.glob(glob_dir)] #通过通配符读取图像文件并且进行resize
# paths = [file for file in glob.glob(glob_dir)] #读取对应的路径
# # images = np.array(images)/255 #将一张图flatten之后归一化

In [20]:
test_input_dir = "./result/temp_data/?/"
test_images,test_paths = readImgAPath(test_input_dir,100,1)
test_labels = [item.split(os.path.sep)[-2] for item in test_paths]
# images_divert = [cv.bitwise_not(item) for item in test_images]  #反转图像黑白转换

In [12]:
len(test_images),test_images[0].shape

(3221, (100, 100, 3))

In [38]:
hits = list()
shutil.rmtree('./result/tess/error') #递归删除文件夹
os.makedirs("./result/tess/error")
temp = test_images
# for i,img in tqdm(enumerate(temp)):
for i in tqdm(range(len(temp))):
    img = temp[i]
    label = test_labels[i]
    pred = tess.image_to_string(img, config=(
                  "-c tessedit"
                  "_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
                  " --psm 10"
                  " -l osd"
                  " "))
    if (label == pred[0]):
        hits.append(1)
    else:
        hits.append(0)
        filename = "./result/tess/error/label is "+label+"pred is "+ pred[0]+".png"
        cv.imwrite(filename,img)    

 49%|██████████████████████████████████████▍                                       | 1461/2964 [03:06<03:11,  7.84it/s]


IndexError: string index out of range

In [ ]:
print("Accuracy: ", sum(hits)/len(hits))

## 处理 # I O文件夹

In [10]:
Pinput_dir = "./result/raw_car_train/1~/"
Pimages,Ppaths = readImgAPath(Pinput_dir)
Pfilenames = [item.split(os.path.sep)[-1] for item in Ppaths]
for i in range(len(Pimages)):
    img = Pimages[i]
    npImg = np.array(img)
    npImg[npImg==255] = 0
    npImg[(npImg<255)&(npImg>0)] = 255
    cv.imwrite("./result/label_car_train_binary/1Process/"+Pfilenames[i],npImg)

### 处理训练集

In [5]:
input_dir = "./result/car_dataset/?/"
images,paths = readImgAPath(input_dir)
labels = [item.split(os.path.sep)[-2] for item in paths]

In [10]:
'''
读取文件夹下所有图片，将其二值化
高斯核去掉噪音，然后使用OTSU算法二值化，再写入文件夹中
输入：需要处理的图像list
输出：二值化之后的图像list
'''
def binary_img(images,lables):
    Gimg_list = []
    for i in range(len(images)):
        img = images[i]
        #img=cv2.imread('F:cat.jpg',0)flag默认为1，即读取为彩色图像，如果要读取为灰度图（单通道），只需要把flag修改为为0即可。可直接读取灰度图
        blur = cv.GaussianBlur(img,(5,5),0)
        ret,thImg = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)  #修改THRESH_BINARY
#         mask = cv.erode(thImg.astype('uint8'), kernel=np.ones((3,3)))
        Gimg_list.append(thImg)
        
        #保存至文件夹
        Ldir = "./result/carBinary/"+lables[i]
        isExists=os.path.exists(Ldir)
        if not isExists:
            os.makedirs(Ldir)
        cv.imwrite(Ldir+"/"+str(i)+".png",thImg)
    return Gimg_list

In [11]:
Gimg_list = binary_img(images,labels)

### SVM方法

In [39]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
# digits = datasets.load_digits() # 加载数据
# # 把数据所代表的图片显示出来
# images_and_labels = list(zip(digits.images, digits.target))
# plt.figure(figsize=(8, 6))
# for index, (image, label) in enumerate(images_and_labels[:8]):
#     plt.subplot(2, 4, index + 1)
#     plt.axis('off')
#     plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
#     plt.title('Digit: %i' % label, fontsize=20);


In [24]:
images_flatten = np.array(np.float32(Gimg_list).reshape(len(Gimg_list),-1)/255)
test_images_flatten = np.array(np.float32(test_images).reshape(len(test_images),-1)/255)

In [40]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(images_flatten, labels, test_size=0.20, random_state=2)
clf = svm.SVC(gamma=0.001, C=100., probability=True)
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)
accuracy_score(Ytest, Ypred)

0.9891891891891892

In [41]:
test_pred = clf.predict(test_images_flatten)
accuracy_score(test_labels, test_pred)

0.2598571872089413

In [29]:
clf = svm.SVC()
clf.fit(images_flatten,labels)
print(clf.score(test_images_flatten,test_labels))

0.34119838559453586

In [48]:
np.set_printoptions(threshold=np.inf) #显示所有内容
test_pred

array(['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '#', '#', '1', '#', '#', '#', '#', '#', '#', '1', '#', '1',
       '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '#', '#', '#', '#', '#', '#', '1', '#', '#', '#', '#', '#',
       '#', '#', '1', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '1', '#', '#', '#', '#', '#', '#', '#', '#', '#', '1', '#',
       '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '1',
       '1', '1', '1', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
       '#', '1', '#', '#', '1', '#', '#', '1', '#', '#', '#', '#', '1',
       '#', '1', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#

In [21]:
#使用tesseract和template 目前无法正确显示
# text_list = []
# for image in images_divert:
#     text = tess.image_to_string(image)
# #     print(text)
#     text_list.append(text)
# template = ['0','1','2','3','4','5','6','7','8','9','/',
#             'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

### 神经网络

### 分类原图标签

In [63]:
test_input_dir = "./result/output10_clean/*/"
test_images,test_paths = readImgAPath(test_input_dir,100)
test_labels = [item.split(os.path.sep)[-2] for item in test_paths]
test_Imgname = [item.split(os.path.sep)[-1] for item in test_paths]

In [65]:
origin_input_dir = "./result/all_rotate_result_big/"
origin_images,origin_paths = readImgAPath(origin_input_dir,100,1)
# origin_labels = [item.split(os.path.sep)[-2] for item in origin_paths]
origin_Imgname = [item.split(os.path.sep)[-1] for item in origin_paths]

In [68]:
for i in range(len(origin_images)):
    img = origin_images[i]
    imgName = origin_Imgname[i]
    Lindex = test_Imgname.index(imgName)
    imgLabel = test_labels[Lindex]
#     print(imgLabel)
    #保存至文件夹
    Ldir = "./result/origin_result/"+imgLabel
    isExists=os.path.exists(Ldir)
    if not isExists:
        os.makedirs(Ldir)
    cv.imwrite(Ldir+"/"+str(imgName)+".png",img)